In [3]:
from utils import Config, ConfigObject

a = Config(cfg_dict={'fruit':{'name': 'apple', 'price': 100, "props":{"weight": 10, "color": "red"}}})
print(a.fruit.name(), a.fruit.price())

attr_list = list(a.fruit.__dict__.keys())
print(attr_list)
print(type(getattr(a.fruit, attr_list[1])))
print(a.to_dict())
b = Config(cfg_dict=a.to_dict())
print(b.fruit.price(), b.fruit.props.color())

apple 100
['val', 'name', 'price', 'props']
<class 'utils.cfg.ConfigObject'>
{'fruit': {'name': 'apple', 'price': 100, 'props': {'weight': 10, 'color': 'red'}}}
100 red


In [7]:
c = b.fruit.props
print(c.color())

red


In [4]:
from utils import Config, ConfigObject
cfg = Config("./cfgs/base.yaml")
print(b.fruit.to_dict())

{'name': 'apple', 'price': 100, 'props': {'weight': 10, 'color': 'red'}}
